In [1]:
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
tokenizer = open_clip.get_tokenizer('hf-hub:imageomics/bioclip')

/home/akeresh/anaconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
open_clip_pytorch_model.bin: 100%|██████████| 599M/599M [00:55<00:00, 10.7MB/s] 
open_clip_config.json: 100%|██████████| 469/469 [00:00<00:00, 53.9kB/s]


In [22]:
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [11]:
image = Image.open("Ursus-arctos.jpeg")
preprocessed_image = preprocess_val(image).unsqueeze(0)

In [32]:
text = tokenizer(["kodiak bear", "a dog", "a cat"])

In [33]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(preprocessed_image)
    norm_features = image_features / image_features.norm(dim=-1, keepdim=True)

    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * norm_features @ text_features.T).softmax(dim=-1)

In [34]:
print("Label probs:", text_probs) 

Label probs: tensor([[9.9996e-01, 4.4641e-05, 2.3945e-07]])


In [18]:
image_features.shape, norm_features.shape

(torch.Size([1, 512]), torch.Size([1, 512]))

In [ ]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

----